In [ ]:
# !pip install vit-pytorch

In [5]:
!pip install einops

In [7]:
import torch
# from vit_pytorch import ViT, MAE
from glob import glob
from torch.utils.data import Dataset, DataLoader
import os
from skimage import io, transform
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import torchvision
from torchvision import transforms
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from torch import nn
import torch.nn.functional as F
from einops import rearrange, repeat
from einops.layers.torch import Rearrange
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt


In [19]:
from skimage import io
im = io.imread('../input/neuron/train-input.tif')
print(im.shape)

In [23]:
plt.imshow(cv2.resize(d_imag, dsize=(256, 256), interpolation=cv2.INTER_CUBIC))

In [ ]:
image = pd.DataFrame(image)

In [ ]:
train, val = train_test_split(image, test_size=0.2)

In [ ]:
# img = Image.open("../input/imagenetmini-1000/imagenet-mini/train/n01440764/n01440764_11170.JPEG")

In [ ]:
# np.asarray(img).shape

In [ ]:
# image = np.load("./seperated/train-input2.npy")
# image = np.resize(image, (256, 256))

In [ ]:
# image = image.reshape((256,256,1))

In [ ]:
# np.stack((image, image, image), axis=2).shape

In [ ]:
# image.shape

In [27]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
class FaceLandmarksDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self,input_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.image_list = input_dir
#         self.landmarks_frame = pd.read_csv(csv_file)
#         self.root_dir = root_dir
        self.transform = transforms.Compose([
                                               transforms.ToTensor(),
                                           ])

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.image_list.iloc[idx, 0])
        
        image = np.load(img_name)
        image = cv2.resize(image, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
#         image = np.resize(image, (256, 256))
#         image = image.reshape((256,256,1))
#         image = np.stack((image, image, image), axis=2)
#         print(np.asarray(image).shape)
#         image = io.imread(img_name)
#         image = np.asarray(image)
#         image = (image - np.amin(image))-(np.amax(image)-np.amin(image)+0.0001)
#         landmarks = self.landmarks_frame.iloc[idx, 1:]
#         landmarks = np.array([landmarks])
#         landmarks = landmarks.astype('float').reshape(-1, 2)
#         sample = {'image': image, 'landmarks': landmarks}

        if self.transform:
            image = self.transform(image)
            
#         print(image.shape)
        return image

## 2D Slice Loader (Randomly)

In [37]:
class SliceDataLoader(Dataset):
    def __init__(self,input_vol,size, transform=None):
        
        self.vol = input_vol
        self.length = size
        self.transform = transforms.Compose([
                                                transforms.ToTensor(),    
                                           ])
        self.randomlist = pd.DataFrame(np.concatenate((np.random.randint(0,768,size=(self.length, 2)), np.random.randint(0,100,size=(self.length, 1))), axis=1))
        
    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        x, y, z = self.randomlist.iloc[idx]
        image = self.vol[z, x:x+256, y:y+256]
        
        if self.transform:
            image = self.transform(image)
            
        return image

In [9]:
lis = pd.DataFrame(np.concatenate((np.random.randint(0,768,size=(50, 2)), np.random.randint(0,100,size=(50, 1))), axis=1))

In [18]:
x, y , _ = lis.iloc[1]

In [38]:
# train_set = FaceLandmarksDataset(image)
train_set = SliceDataLoader(im, 1000)
val_set = SliceDataLoader(im, 100)
# val_set = FaceLandmarksDataset(glob("../input/imagenetmini-1000/imagenet-mini/val/*/*.JPEG"))

In [39]:
BATCH_SIZE = 1
WORKERS = 0
SHUFFLE = True
train_loader = DataLoader(train_set,  batch_size=BATCH_SIZE,
                        shuffle=SHUFFLE, num_workers=WORKERS)
val_loader = DataLoader(val_set,  batch_size=BATCH_SIZE,
                        shuffle=SHUFFLE, num_workers=WORKERS)

In [40]:
# helpers

def pair(t):
    return t if isinstance(t, tuple) else (t, t)

# classes

class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn
    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

class Attention(nn.Module):
    def __init__(self, dim, heads = 8, dim_head = 64, dropout = 0.):
        super().__init__()
        inner_dim = dim_head *  heads
        project_out = not (heads == 1 and dim_head == dim)

        self.heads = heads
        self.scale = dim_head ** -0.5

        self.attend = nn.Softmax(dim = -1)
        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

    def forward(self, x):
        qkv = self.to_qkv(x).chunk(3, dim = -1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = self.heads), qkv)

        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale

        attn = self.attend(dots)

        out = torch.matmul(attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)

class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout = 0.):
        super().__init__()
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(dim, Attention(dim, heads = heads, dim_head = dim_head, dropout = dropout)),
                PreNorm(dim, FeedForward(dim, mlp_dim, dropout = dropout))
            ]))
    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x
        return x

class ViT(nn.Module):
    def __init__(self, *, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, pool = 'cls', channels = 1, dim_head = 64, dropout = 0., emb_dropout = 0.):
        super().__init__()
        image_height, image_width = pair(image_size)
        patch_height, patch_width = pair(patch_size)
        print(patch_height, patch_width)

        assert image_height % patch_height == 0 and image_width % patch_width == 0, 'Image dimensions must be divisible by the patch size.'

        num_patches = (image_height // patch_height) * (image_width // patch_width)
        patch_dim = channels * patch_height * patch_width
        assert pool in {'cls', 'mean'}, 'pool type must be either cls (cls token) or mean (mean pooling)'

        self.to_patch_embedding = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = patch_height, p2 = patch_width),
            nn.Linear(patch_dim, dim),
        )

        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.dropout = nn.Dropout(emb_dropout)

        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)

        self.pool = pool
        self.to_latent = nn.Identity()

        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes)
        )

    def forward(self, img):
        x = self.to_patch_embedding(img)
        
        b, n, _ = x.shape
    
        cls_tokens = repeat(self.cls_token, '() n d -> b n d', b = b)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding[:, :(n + 1)]
        x = self.dropout(x)

        x = self.transformer(x)

        x = x.mean(dim = 1) if self.pool == 'mean' else x[:, 0]

        x = self.to_latent(x)
        return self.mlp_head(x)

In [41]:
v = ViT(
    image_size = 256,
    patch_size = 8,
    num_classes = 1000,
    dim = 1024,
    depth = 6,
    heads = 8,
    channels=1,
    mlp_dim = 2048
)

if torch.cuda.is_available():
    v = v.cuda()

In [42]:
class MAE(nn.Module):
    def __init__(
        self,
        *,
        encoder,
        decoder_dim,
        masking_ratio = 0.75,
        decoder_depth = 1,
        decoder_heads = 8,
        decoder_dim_head = 64
    ):
        super().__init__()
        assert masking_ratio > 0 and masking_ratio < 1, 'masking ratio must be kept between 0 and 1'
        self.masking_ratio = masking_ratio

        # extract some hyperparameters and functions from encoder (vision transformer to be trained)
        
        self.encoder = encoder
        num_patches, encoder_dim = encoder.pos_embedding.shape[-2:]
        self.to_patch, self.patch_to_emb = encoder.to_patch_embedding[:2]  ## seperating the patch embedding list
        pixel_values_per_patch = self.patch_to_emb.weight.shape[-1] ##3072

        # decoder parameters

        self.enc_to_dec = nn.Linear(encoder_dim, decoder_dim) if encoder_dim != decoder_dim else nn.Identity()
        self.mask_token = nn.Parameter(torch.randn(decoder_dim))
        self.decoder = Transformer(dim = decoder_dim, depth = decoder_depth, heads = decoder_heads, dim_head = decoder_dim_head, mlp_dim = decoder_dim * 4)
        self.decoder_pos_emb = nn.Embedding(num_patches, decoder_dim)
        self.to_pixels = nn.Linear(decoder_dim, pixel_values_per_patch)
#         self.to_image = nn.Sequential(
#             Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = patch_height, p2 = patch_width),
#             nn.Linear(patch_dim, dim),
#         )

    def forward(self, img):
        device = img.device

        # get patches

        patches = self.to_patch(img) #convert image to patches [100, 64, 3072]
#         print(patches.shape, img.shape)
        batch, num_patches, *_ = patches.shape
        #batches = 100, num_patches = 64

        # patch to encoder tokens and add positions

        tokens = self.patch_to_emb(patches) #change num_patches to dim i.e from 3072 to 1024
        #(100,64,1024)
        tokens = tokens + self.encoder.pos_embedding[:, 1:(num_patches + 1)] # (1, 64, 1024) of pos_embedding is added to token
#         print(1, (self.encoder.pos_embedding.shape),(self.encoder.pos_embedding[:, 1:(num_patches + 1)].shape))
        # add embedded pathces to ;to randomly initialised positional embeddings
        #(100,64,1024)

        # calculate of patches needed to be masked, and get random indices, dividing it up for mask vs unmasked

        num_masked = int(self.masking_ratio * num_patches) #patches that has to masked 
        rand_indices = torch.rand(batch, num_patches, device = device).argsort(dim = -1) # indices are arranged in by the order in which they are appearing
        #rand_indices shape is (batch, num_patches) i.e. 100,64 
        masked_indices, unmasked_indices = rand_indices[:, :num_masked], rand_indices[:, num_masked:]
        #masked indices = (batch, masked), unmasked indices = (batch, unmasked)
        # get the unmasked tokens to be encoded
#         print(masked_indices)
        batch_range = torch.arange(batch, device = device)[:, None]
        tokens = tokens[batch_range, unmasked_indices]
#         print(tokens)
        # get the patches to be masked for the final reconstruction loss

        masked_patches = patches[batch_range, masked_indices]
#         print(1, masked_patches.shape) 

        # attend with vision transformer

        encoded_tokens = self.encoder.transformer(tokens)
        
        # project encoder to decoder dimensions, if they are not equal - the paper says you can get away with a smaller dimension for decoder

        decoder_tokens = self.enc_to_dec(encoded_tokens)

        # repeat mask tokens for number of masked, and add the positions using the masked indices derived above

        mask_tokens = repeat(self.mask_token, 'd -> b n d', b = batch, n = num_masked)
        mask_tokens = mask_tokens + self.decoder_pos_emb(masked_indices)

        # concat the masked tokens to the decoder tokens and attend with decoder

        decoder_tokens = torch.cat((mask_tokens, decoder_tokens), dim = 1)
        decoded_tokens = self.decoder(decoder_tokens)

        # splice out the mask tokens and project to pixel values

        mask_tokens = decoded_tokens[:, :num_masked]
        pred_pixel_values = self.to_pixels(mask_tokens)
        return pred_pixel_values, masked_patches, patches, masked_indices

#         calculate reconstruction loss

#         recon_loss = F.mse_loss(pred_pixel_values, masked_patches)
#         return recon_loss

In [43]:

mae = MAE(
    encoder = v,
    masking_ratio = 0.75,   # the paper recommended 75% masked patches
    decoder_dim = 512,      # paper showed good results with just 512
    decoder_depth = 6       # anywhere from 1 to 8
)
if torch.cuda.is_available():
    mae = mae.cuda()

In [44]:
criterion = nn.MSELoss()
# if torch.cuda.is_available():
#     criterion = criterion.cuda()
optimizer = torch.optim.Adam(mae.parameters(), lr=0.001)

In [ ]:
var = np.empty(())

In [45]:
# loss = mae(images)
# # loss.backward()
pred = np.array
patchess = np.array

for epoch in range(300):  # loop over the dataset multiple times
    i=0
    running_loss = 0.0
    train_loss = 0.0
    for data in tqdm(train_loader):
        # get the inputs; data is a list of [inputs, labels]
        data = data.to(device)
        images = data
        i+=1

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
#         outputs = net(inputs)
#         print(images.shape)
#         print(images.shape)
        pred, masked,patchess, indices = mae(images)
#         print(pred.shape)
        loss = criterion(pred, masked)
        loss.backward()
        optimizer.step()
        if epoch%60==0:
            torch.save(v.state_dict(), './trained-vit'+str(epoch)+'.pt')
            torch.save(mae, "./model"+str(epoch))
        # print statistics
        running_loss += loss.item()
    print('[%d] loss: %.3f' %
            (epoch + 1, running_loss))
    
    for data in tqdm(val_loader):
        # get the inputs; data is a list of [inputs, labels]
        data = data.to(device)
        images = data
        i+=1


        pred, masked,patchess, indices = mae(images)
        loss = criterion(pred, masked)
        if epoch%60==0:
            torch.save(v.state_dict(), './trained-vit'+str(epoch)+'.pt')
            torch.save(mae, "./model"+str(epoch))
        # print statistics
        running_loss += loss.item()
        
    print('[%d] loss: %.3f' %
            (epoch + 1, train_loss))

print('Finished Training')
# that's all!
# do the above in a for loop many times with a lot of images and your vision transformer will learn

# save your improved vision transformer
torch.save(v.state_dict(), './trained-vit.pt')

In [ ]:
you = Rearrange('b (h w) (p1 p2 c) -> b c (h p1) (w p2)', h=32, w = 32, p1 = 8, p2 = 8, c = 1)

In [ ]:
orig = patchess.cpu().detach().numpy()
orig1 = you(orig)

In [ ]:

# orig = orig.reshape((100, 1, 256,256))

plt.imshow(orig1[0][0], cmap='gray')
np.save("./original.npy",orig1)

In [ ]:
batch_range = torch.arange(BATCH_SIZE)[:, None]
patchess[batch_range, indices] = pred

In [ ]:
imga = patchess.cpu().detach().numpy()
imga = you(imga)
# img = imga.reshape((100, 1, 256,256))
plt.imshow(imga[0][0], cmap='gray')
np.save("./outputImage.npy",imga)

In [ ]:
# running_loss*2

In [ ]:
# for data in train_loader:
# #     print(data[0].shape)
#     image = np.array(data[0])
#     print(image.shape)
# #     image = image.reshape((256,256,3))
# #     image = Image.fromarray(image)
#     im = Image.fromarray(np.uint8(image*255))
#     im.save("./file.jpeg" + ".jpg", "JPEG")